In [1]:
import pandas as pd
import numpy as np
import pyarrow
import fastparquet


import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import col,lpad, length, lit, to_date,to_timestamp, when, date_format, coalesce, md5, sha2, concat_ws,trim

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Qlik_transfo") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 15:06:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path_ = "/Users/dataworkshop/Desktop/Spark/Frais de gestion/Table de param & ds/datasetV1.xlsx"

file_path = "//Users/dataworkshop/Desktop/Spark/Frais de gestion/Table de param & ds/Frasi_Gestion.xlsx"
Suivi_et_Archive = "Suivi_et_Archive"
Tab_PARAM = "Tab_PARAM"
PARAM_FG = "PARAM_FG"

In [4]:
df_Suivi_et_Archive = pd.read_excel(file_path_, header=1)
df_Tab_PARAM = pd.read_excel(file_path, Tab_PARAM)
df_PARAM_FG = pd.read_excel(file_path, PARAM_FG)

Param FG

In [5]:
# effecaer les colonnes qui sont vides
columns_to_delete = ['Unnamed: 9', 'Unnamed: 12',"Unnamed: 16",]
# Effacer les colonnes vide
df_PARAM_FG = df_PARAM_FG.drop(columns=columns_to_delete)


In [6]:
schema_PARAM_FG = StructType([
    StructField('Unnamed: 0', StringType(), True),
    StructField('Année', StringType(), True),
    StructField('Catégorie', StringType(), True),
    StructField('Sous-Catégorie',StringType(), True),
    StructField('Recueillir les candidatures TTC', IntegerType(), True),
    StructField('Instruire les dossiers TTC',IntegerType(), True),
    StructField('Accompagner les projets TTC', IntegerType(), True),
    StructField('Recueillir les candidatures HT',IntegerType(), True),
    StructField('Instruire les dossiers HT', IntegerType(), True),
    StructField('Accompagner les projets HT',IntegerType(), True),
    StructField('Reporting et évaluation TTC', IntegerType(), True),
    StructField('Reporting et évaluation HT',IntegerType(), True),
    StructField('Convention SCAN', StringType(), True),
    StructField('Convention', StringType(), True),
    StructField('Sous-fonds FG AAP/Projets',StringType(), True),
    StructField('Dispositif', StringType(), True),
    StructField('Dispositif Cible', StringType(), True),
    StructField('AAP', StringType(), True),
    StructField('Date de relève',StringType(), True),
    StructField('Acronyme du projet', StringType(), True),
    StructField("Nom de l'entreprise", StringType(), True),
    StructField("Identifiant projet",StringType(), True),
    StructField("Montant total de l'aide retenue HT", IntegerType(), True),
    StructField('COPIL : Date de signature DPM',StringType(), True),
    StructField('Convention.1', StringType(), True),
    StructField('AAP_Gré-à-gré', StringType(), True),])



# MAPPING
PARAM_FG = spark.read.csv('df_PARAM_FG_clean.csv', header=True, schema=schema_PARAM_FG)

# Replace NaN values with None
PARAM_FG = PARAM_FG.na.replace(float('nan'), None)

In [7]:
df = spark.createDataFrame(pd.read_csv('dataset_version1.csv').replace({float('nan'): None}))

/var/folders/6d/b6p57_cn0t9g6b9c6t40smg80000gn/T/ipykernel_16111/3874427650.py:1: DtypeWarning: Columns (77,78,79,80,109,112,121,122,123,124,126) have mixed types. Specify dtype option on import or set low_memory=False.
  df = spark.createDataFrame(pd.read_csv('dataset_version1.csv').replace({float('nan'): None}))
/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [8]:
Param = df_Tab_PARAM[['Statut', "Terminé/En cours",'Liste_Relèves',"Stade_Relèves_FG"]]

Param = spark.createDataFrame(Param)
# rename stat to be able to map terminé/en cours properly
Param = Param.withColumnRenamed("Statut", "Statut_")

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


lire fichier param_cadrage

In [9]:
# Specify the path to your Excel file
excel_file_path = "/Users/dataworkshop/Desktop/Spark/FG_PARAM.xlsx"

# Read the Excel file into a Pandas DataFrame
FG_Param = pd.read_excel(excel_file_path)

# Convert the Pandas DataFrame to a PySpark DataFrame
FG_Param = spark.createDataFrame(FG_Param)

FG_Param = FG_Param.withColumnRenamed("AAP", "_AAP")


/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [10]:
from pyspark.sql import functions as F

# Les données à ramener du dataset


F_G_AAP_Phase_Cadrage = df.select("AAP", "Dispositif", "Date de relève","Identifiant projet")

# Adjust AAP Nuclaire and all AAP's

#delarer une vairable de table de param
aap_mapping_value = PARAM_FG[["Identifiant projet","AAP_Gré-à-gré"]]

# rename ID_projet to get rid of ambiguity
aap_mapping_value = aap_mapping_value.withColumnRenamed("Identifiant projet", "Identifiant projet_")



# Alias the DataFrames
df1 = F_G_AAP_Phase_Cadrage.alias("df1")
df2 = aap_mapping_value.alias("df2")

# Join with alias and specify the columns with qualified names
F_G_AAP_Phase_Cadrage = df1.join(df2, df1["Identifiant projet"] == df2["Identifiant projet_"], "left_outer")

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn(
    "AAP",
    when(col("AAP").startswith("Gré-à-gré Etat"), col("AAP_Gré-à-gré"))
    .otherwise(F_G_AAP_Phase_Cadrage["AAP"]))

# supprimer les deux colonnes ID_projet
#F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet")
#F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet_")



# Renommer date de relève

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumnRenamed("Date de relève", "1ere année de relève")


# Mapper Catégorie AAP

FG_Param_ =FG_Param[["_AAP","Catégorie_AAP"]]

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(FG_Param_, F_G_AAP_Phase_Cadrage["AAP"] == FG_Param_["_AAP"], "left_outer")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("_AAP")

# Mapper convention 

FG_Param1 =FG_Param[["_AAP","Convention_AAP"]]

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.join(FG_Param1, F_G_AAP_Phase_Cadrage["AAP"] == FG_Param1["_AAP"], "left_outer")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("_AAP")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumnRenamed("Convention_AAP", "Convention")


# Col Cadrer la procedure - AAP renouvelé

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", when((F_G_AAP_Phase_Cadrage["AAP"].substr(-1, 1) != "1") & (F_G_AAP_Phase_Cadrage["AAP"].substr(-1, 1) != "2"), 5500))
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure - AAP renouvelé", col("Cadrer la procédure - AAP renouvelé").cast("int"))

# Col Cadrer la procedure 

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", when(F_G_AAP_Phase_Cadrage["Cadrer la procédure - AAP renouvelé"].isNull(), lit(11167)))
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn("Cadrer la procédure", col("Cadrer la procédure").cast("int"))

# les exceptions de gré-à-gré

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.withColumn('Cadrer la procédure', when(col('Identifiant projet_').isNotNull(), 33583).otherwise(col('Cadrer la procédure')))

#F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("Identifiant projet_")
F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.drop("AAP_Gré-à-gré")

# get distinct
F_G_AAP_Phase_Cadrage.distinct()

# plusieurs colonnes de Reporrting & Evaluation 

# ramener le montant qu'on va diviser
MT_Reporting_evaluation = PARAM_FG.filter(col("Reporting et évaluation HT").isNotNull())
MT_Reporting_evaluation = MT_Reporting_evaluation[["Reporting et évaluation HT"]]
MT_Reporting_evaluation = MT_Reporting_evaluation.withColumn("Reporting et évaluation HT", col("Reporting et évaluation HT").cast("int"))

# ramner les données de parametrage pour identifier les AAP sur les quel on va diviser le montant de reporting evaluation


file_path = "/Users/dataworkshop/Desktop/Spark/Frais de gestion/Jobs/Table FG paramètrage frais reporting propal 2.xlsx"
AAP = "AAP"

# transform excel in to dataframe
df_reporting_annees = pd.read_excel(file_path,AAP, header=0)
param_csv =df_reporting_annees.to_csv("df_reporting_annees.csv")
df_reporting_annees = spark.createDataFrame(pd.read_csv('df_reporting_annees.csv').replace({float('nan'): None}))
# show the data

/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [28]:
F_G_AAP_Phase_Cadrage.show()

24/01/04 15:08:52 WARN TaskSetManager: Stage 252 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+----------+---------------+--------------------+--------------------+-------------------+-----------------------------------+
|                 AAP|Dispositif|  Catégorie_AAP|          Convention|1ere année de relève|Cadrer la procédure|Cadrer la procédure - AAP renouvelé|
+--------------------+----------+---------------+--------------------+--------------------+-------------------+-----------------------------------+
|                NULL| 30 BIOMED|           NULL|                NULL|               45219|              11167|                               NULL|
|                2021|     I-LAB|          i-Lab|          Structurel| 2021-02-02 00:00:00|              11167|                               NULL|
|                2022|     I-LAB|          i-Lab|          Structurel| 2022-02-01 00:00:00|              11167|                               NULL|
|                2023|     I-LAB|          i-Lab|          Structurel| 2023-02-01 00:00:00|               NULL| 

24/01/04 16:05:19 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 413403 ms exceeds timeout 120000 ms
24/01/04 16:05:19 WARN SparkContext: Killing executors is not supported by current scheduler.
24/01/04 16:05:20 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
import datetime
from pyspark.sql.functions import round


# Assuming you have a SparkSession named 'spark'
spark = SparkSession.builder.appName("your_app_name").getOrCreate()

def calculate_result_for_year(df_reporting_annees, MT_Reporting_evaluation, year):
    # Filter rows where the column for the specific year is 1
    df_reporting_annees_year = df_reporting_annees.filter(col(f'AAP Actifs {year}') == 1)
    
    # Count the occurrences
    count_of_ones = df_reporting_annees_year.select('AAP').distinct().count()
    
    # Get the integer value from the second table
    reporting_evaluation_value = MT_Reporting_evaluation.select("Reporting et évaluation HT").collect()[0][0]
    
    # Divide the integer value by the total number of active AAPs for the specific year
    result = reporting_evaluation_value / count_of_ones
    
    # Update the DataFrame with the calculated result for the specific year
    df_reporting_annees = df_reporting_annees.withColumn(f'AAP Actifs {year}', when(col(f'AAP Actifs {year}') == 1, result).otherwise(None))
    
    return df_reporting_annees

# Get the current year
current_year = datetime.datetime.now().year

# Calculate results for each year from 2021 to the current year
for year in range(2021, current_year + 1):
    df_reporting_annees = calculate_result_for_year(df_reporting_annees, MT_Reporting_evaluation, year)

columns_to_drop = ["Unnamed: 0","Catégorie_AAP", "Convention_AAP"]
df_reporting_annees = df_reporting_annees.drop(*columns_to_drop)


# arrandire les AAP actifs

# Voici comment tu peux arrondir toutes ces colonnes
colonnes_a_arrondir = ["AAP Actifs 2022", "AAP Actifs 2023", "AAP Actifs 2024", "AAP Actifs 2025", "AAP Actifs 2026",
                       "AAP Actifs 2027", "AAP Actifs 2028", "AAP Actifs 2029", "AAP Actifs 2030", "AAP Actifs 2031",
                       "AAP Actifs 2032", "AAP Actifs 2033", "AAP Actifs 2034", "AAP Actifs 2035"]

# Appliquer la fonction round à chaque colonne
for colonne in colonnes_a_arrondir:
    df_reporting_annees = df_reporting_annees.withColumn(colonne, round(colonne, 0))

# Show the updated DataFrame

24/01/04 15:06:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
from pyspark.sql.functions import col, sum, min, max, first

# Display the similar version of 

F_G_AAP_Phase_Cadrage = F_G_AAP_Phase_Cadrage.groupBy("AAP").agg(
    first("Dispositif").alias("Dispositif"),
    first("Catégorie_AAP").alias("Catégorie_AAP"),
    first("Convention").alias("Convention"),
    min("1ere année de relève").alias("1ere année de relève"),
    max("Cadrer la procédure").alias("Cadrer la procédure"),
    max("Cadrer la procédure - AAP renouvelé").alias("Cadrer la procédure - AAP renouvelé"),
    
    
)

# Show the result
F_G_AAP_Phase_Cadrage.show(100)

24/01/04 15:06:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/01/04 15:06:26 WARN TaskSetManager: Stage 35 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------+---------------+--------------------+--------------------+-------------------+-----------------------------------+
|                 AAP|    Dispositif|  Catégorie_AAP|          Convention|1ere année de relève|Cadrer la procédure|Cadrer la procédure - AAP renouvelé|
+--------------------+--------------+---------------+--------------------+--------------------+-------------------+-----------------------------------+
|                NULL|     30 BIOMED|           NULL|                NULL|               45219|              11167|                               NULL|
|                2021|         I-LAB|          i-Lab|          Structurel| 2021-02-02 00:00:00|              11167|                               NULL|
|                2022|         I-LAB|          i-Lab|          Structurel| 2022-02-01 00:00:00|              11167|                               NULL|
|                2023|         I-LAB|          i-Lab|          Structurel| 2023-02-01 00

# F_G_projets_Phase_1

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import year
from pyspark.sql.functions import col, sum, min, max, first



F_G_projet_1 = df[["Identifiant projet", 'Dispositif',"AAP", 'Acronyme du projet', 'Instruction affectée autre opérateur', 
                 'COPIL : Date de signature DPM',"Montant total de l'aide retenue HT", 'Date de verrouillage',"COPIL : Décision Comité"]]

# str to int

F_G_projet_1 = F_G_projet_1.withColumn("Montant total de l'aide retenue HT", col("Montant total de l'aide retenue HT").cast("int"))


#-------------------------------------------------------------------------------------------------------------------------------------------------------
# renommé 'Dispositif / Opérateur cible' ==> Dispositif de sortie
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Dispositif", "Dispositif de sortie")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Catégorie Dispositif  ==> Convention AAP dans F_G_AAP_Phase_Cadrage

FG_Param1 =FG_Param[["_AAP","Convention_AAP"]]
F_G_projet_1 = F_G_projet_1.join(FG_Param1, F_G_projet_1["AAP"] == FG_Param1["_AAP"], "left_outer")
F_G_projet_1 = F_G_projet_1.drop("_AAP")

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Convention AAP ==> convention dans F_G_AAP_Phase_Cadrage
FG_Param2 =FG_Param[["_AAP","Catégorie_AAP"]]
F_G_projet_1 = F_G_projet_1.join(FG_Param2, F_G_projet_1["AAP"] == FG_Param2["_AAP"], "left_outer")
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Catégorie_AAP", "Catégorie Dispositif")
F_G_projet_1 = F_G_projet_1.drop("_AAP")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# sous-fonds FG == 

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# renommé Instruction affectée autre opérateur ==> Instruction autre opérateur
F_G_projet_1 = F_G_projet_1.withColumnRenamed("Instruction affectée autre opérateur", "Instruction autre opérateur")
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de projet Décidés COPIL ==> regarder la RG dans neurone   " Une histoire de date de cloture "

condition_nb_projet_decide = col("COPIL : Décision Comité") == 'Favorable'
F_G_projet_1 = F_G_projet_1.withColumn("Nombre de projet décidés COPIL", when(condition_nb_projet_decide, 1).otherwise(None))
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de partenaire Décidés COPIL ==> regarder la RG dans neurone   " Une histoire de date de cloture "
# cette colonne sera créer dans le groupby

# Assuming F_G_projet_1 is your DataFrame
F_G_Nombre_patrtenaire = F_G_projet_1.groupBy("Identifiant projet").agg(
    F.sum("Nombre de projet décidés COPIL").alias("Nombre de partenaire décidés COPIL")
)
F_G_Nombre_patrtenaire = F_G_Nombre_patrtenaire.withColumnRenamed("Identifiant projet", "Identifiant projet_")

F_G_projet_1 = F_G_projet_1.join(F_G_Nombre_patrtenaire, F_G_projet_1["Identifiant projet"] == F_G_Nombre_patrtenaire["Identifiant projet_"], "left_outer")
F_G_projet_1 = F_G_projet_1.drop("Identifiant projet_")


#-------------------------------------------------------------------------------------------------------------------------------------------------------
# créer Nombre de projets verrouillés : Sum of notnull values in date de verrouillage
confition_nb_pr_verrouillés = col("Date de verrouillage").isNotNull()
F_G_projet_1 = F_G_projet_1.withColumn("Nombre de projets verrouilles", when(confition_nb_pr_verrouillés, 1).otherwise(0))

#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Mapper Reveuillir les candidatures deuis Param_FG

receuille_candi =PARAM_FG[["Année","Catégorie","Recueillir les candidatures HT"]]

#format année
receuille_candi = receuille_candi.withColumn('Année', receuille_candi['Année'].cast(IntegerType()))
receuille_candi = receuille_candi.withColumn('Année', receuille_candi['Année'].cast(StringType()))

# Receuillir_key
receuille_candi = receuille_candi.withColumn("Receuillir_key", concat_ws("_", col("Année"), col("Catégorie"))) 
#receuille_candi = receuille_candi.drop("Année")
#receuille_candi = receuille_candi.drop("Catégorie")


# create year 
F_G_projet_1 = F_G_projet_1.withColumn("Date de verrouillage", F.to_date(col("Date de verrouillage")))
F_G_projet_1 = F_G_projet_1.withColumn("Année de verrouillage" , year(F_G_projet_1["Date de verrouillage"]))

F_G_projet_1 = F_G_projet_1.withColumn("Receuillir_key_", concat_ws("_", col("Année de verrouillage"), col("Catégorie Dispositif"))) 


F_G_projet_1 = F_G_projet_1.join(receuille_candi, F_G_projet_1["Receuillir_key_"] == receuille_candi["Receuillir_key"], "left_outer") 

#F_G_projet_1 = F_G_projet_1.drop("Receuillir_key_")
#F_G_projet_1 = F_G_projet_1.drop("Receuillir_key")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------

# Display the similar version of 

F_G_projet_1 = F_G_projet_1.groupBy("Identifiant projet").agg(
    first("Dispositif de sortie").alias("Dispositif de sortie"),
    first("AAP").alias("AAP"),
    first("Acronyme du projet").alias("Acronyme du projet"),
    first("Catégorie Dispositif").alias("Catégorie Dispositif"),
    first("Convention_AAP").alias("Convention AAP"),
    first("Instruction autre opérateur").alias("Instruction autre opérateur"),
    min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
    max("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
    first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
    max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
    first("Date de verrouillage").alias("Date de verrouillage"),    
    first("Nombre de projets verrouilles").alias("Nombre de projets verrouilles"),
    max("Recueillir les candidatures HT").alias("Recueillir les candidatures HT"),
)

# Show the result
F_G_projet_1.show()


24/01/04 15:06:27 WARN TaskSetManager: Stage 38 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
24/01/04 15:06:27 WARN TaskSetManager: Stage 41 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+---------------------------+-----------------------------+----------------------------------+------------------------------+----------------------------------+--------------------+-----------------------------+------------------------------+
|  Identifiant projet|Dispositif de sortie|                 AAP|Acronyme du projet|Catégorie Dispositif|      Convention AAP|Instruction autre opérateur|COPIL : Date de signature DPM|Montant total de l'aide retenue HT|Nombre de projet décidés COPIL|Nombre de partenaire décidés COPIL|Date de verrouillage|Nombre de projets verrouilles|Recueillir les candidatures HT|
+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+---------------------------+-----------------------------+----------------------------------+------------------------------+-----------------

In [14]:
F_G_projet_1.filter(col("Identifiant projet")=="SA 5G-Gré-à-gré Etat-PIC-45524").show(truncate=False)

24/01/04 15:06:28 WARN TaskSetManager: Stage 60 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
24/01/04 15:06:28 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
24/01/04 15:06:28 WARN TaskSetManager: Stage 63 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+------------------------------+--------------------+--------------+------------------+--------------------+--------------+---------------------------+-----------------------------+----------------------------------+------------------------------+----------------------------------+--------------------+-----------------------------+------------------------------+
|Identifiant projet            |Dispositif de sortie|AAP           |Acronyme du projet|Catégorie Dispositif|Convention AAP|Instruction autre opérateur|COPIL : Date de signature DPM|Montant total de l'aide retenue HT|Nombre de projet décidés COPIL|Nombre de partenaire décidés COPIL|Date de verrouillage|Nombre de projets verrouilles|Recueillir les candidatures HT|
+------------------------------+--------------------+--------------+------------------+--------------------+--------------+---------------------------+-----------------------------+----------------------------------+------------------------------+-----------------------

# F_G_Projet_Phase_2_3

Old code 

In [15]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import col
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import translate


# create month & year to transform liste_Relève
df = df.withColumn("Month de relève", date_format("Date de relève", "MMM"))
df = df.withColumn("Year de relève", date_format("Date de relève", "yy"))



# Define your conditions
jan = col("Month de relève") == "Jan"
fev = col("Month de relève") == "Feb"
mar = col("Month de relève") == "Mar"
apr = col("Month de relève") == "Apr"
mai = col("Month de relève") == "May"
jun = col("Month de relève") == "Jun"
july = col("Month de relève") == "Jul"
aug = col("Month de relève") == "Aug"
sep = col("Month de relève") == "Sep"
oct = col("Month de relève") == "Oct"
nov = col("Month de relève") == "Nov"
dec = col("Month de relève") == "Dec"

# Apply the conditions and create Résultat de l'instruction Autres/AMI
df = df.withColumn(
    "Month de relève",
    when(jan, "janv")
    .when(fev, "févr")
    .when(mar, "mars")
    .when(apr, "avr")
    .when(mai, "Mai")
    .when(jun, "juin")
    .when(july, "juil")
    .when(aug, "août")
    .when(sep, "sept")
    .when(oct, "oct")
    .when(nov, "nov")
    .when(dec, "déc")
    .otherwise(col("Month de relève"))
)

# Concatener mois_année

df = df.withColumn("month/year", concat_ws("-", col("Month de relève"), col("Year de relève"))) 
df = df.withColumn("Liste_Relèves_", concat_ws(" - ", col("AAP"), col("month/year")))

In [16]:
F_G_projet_2_3 = df[["Identifiant projet", 'Dispositif', "AAP", 'Acronyme du projet','Statut',
                   'Convention','Instruction affectée autre opérateur','COPIL : Date de signature DPM',
                   'COEX : Date de validation du PV de présélection', 'Date de signature client',
                   "COEX : Décision Comité","Date de relève",'COPIL : Décision Comité',"Montant total de l'aide retenue HT"]]
#________________---------------------------

# Convention AAP ==> convention dans F_G_AAP_Phase_Cadrage
FG_Param2 =FG_Param[["_AAP","Catégorie_AAP"]]
F_G_projet_2_3 = F_G_projet_2_3.join(FG_Param2, F_G_projet_2_3["AAP"] == FG_Param2["_AAP"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Catégorie_AAP", "Catégorie Dispositif")
F_G_projet_2_3 = F_G_projet_2_3.drop("_AAP")



#--1-- Mapper stade projet depuis tab_param & rename it to stade projet

F_G_projet_2_3 = F_G_projet_2_3.join(Param, F_G_projet_2_3["Statut"] == Param["Statut_"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.drop("Statut_")
# Rename Terminé/En cours to Stade
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Terminé/En cours", "Stade projet")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#--2-- créer stade relève à partir de regle de gestion dans neurone


# create month & year to transform liste_Relève
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Month de relève", date_format("Date de relève", "MMM"))
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Year de relève", date_format("Date de relève", "yy"))



# Define your conditions
jan = col("Month de relève") == "Jan"
fev = col("Month de relève") == "Feb"
mar = col("Month de relève") == "Mar"
apr = col("Month de relève") == "Apr"
mai = col("Month de relève") == "May"
jun = col("Month de relève") == "Jun"
july = col("Month de relève") == "Jul"
aug = col("Month de relève") == "Aug"
sep = col("Month de relève") == "Sep"
oct = col("Month de relève") == "Oct"
nov = col("Month de relève") == "Nov"
dec = col("Month de relève") == "Dec"

# Apply the conditions and create Résultat de l'instruction Autres/AMI
F_G_projet_2_3 = F_G_projet_2_3.withColumn(
    "Month de relève",
    when(jan, "janv")
    .when(fev, "févr")
    .when(mar, "mars")
    .when(apr, "avr")
    .when(mai, "Mai")
    .when(jun, "juin")
    .when(july, "juil")
    .when(aug, "août")
    .when(sep, "sept")
    .when(oct, "oct")
    .when(nov, "nov")
    .when(dec, "déc")
    .otherwise(col("Month de relève"))
)

# Concatener mois_année
F_G_projet_2_3 = F_G_projet_2_3.withColumn("month/year", concat_ws("-", col("Month de relève"), col("Year de relève"))) # trim vID_Partenaire
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Liste_Relèves_", concat_ws(" - ", col("AAP"), col("month/year"))) # trim vID_Partenaire


# get the values to create Stade_relève
Param_stade_relève = Param[['Liste_Relèves',"Stade_Relèves_FG"]]
Param_stade_relève = Param_stade_relève.withColumnRenamed("Stade_Relèves_FG", "Stade_Relèves_FG_")

#jointure Stade_relève
F_G_projet_2_3 = F_G_projet_2_3.join(Param_stade_relève, F_G_projet_2_3["Liste_Relèves_"] == Param_stade_relève["Liste_Relèves"], "left")
F_G_projet_2_3 = F_G_projet_2_3.drop("Liste_Relèves")
F_G_projet_2_3 = F_G_projet_2_3.drop("Liste_Relèves_")

F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Stade_Relèves_FG", "Stade Relèves")
F_G_projet_2_3 = F_G_projet_2_3.drop("Stade_Relèves_FG_")


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#--3-- renommer convention à convention projet SCAN
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("convention", "Convention Projet SCAN")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#--4-- Mapper Convention projet pour FG ( Mapper la colonne de Param_FG )
Conv_Map = PARAM_FG[["Convention SCAN", "Convention"]]
#jointure convention
F_G_projet_2_3 = F_G_projet_2_3.join(Conv_Map, F_G_projet_2_3["Convention Projet SCAN"] == Conv_Map["Convention SCAN"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.drop("Convention SCAN")
# rename Convention
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("convention", "Convention Projet pour FG")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#--5-- ramener Nombre de projet Décidés COPIL
ondition_nb_projet_decide = col("COPIL : Décision Comité") == 'Favorable'
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Nombre de projet décidés COPIL", when(ondition_nb_projet_decide, 1).otherwise(None))
#-------------------------------------------------------------------------------------------------------------------------------------------------------
#--6-- ramener Nombre de partenaire Décidés COPIL


F_G_Nombre_patrt = F_G_projet_2_3.groupBy("Identifiant projet").agg(
    F.sum("Nombre de projet décidés COPIL").alias("Nombre de partenaire décidés COPIL")
    )
F_G_Nombre_patrt = F_G_Nombre_patrt.withColumnRenamed("Identifiant projet", "Identifiant projet_")

F_G_projet_2_3 = F_G_projet_2_3.join(F_G_Nombre_patrt, F_G_projet_2_3["Identifiant projet"] == F_G_Nombre_patrt["Identifiant projet_"], "left_outer")
F_G_projet_2_3 = F_G_projet_2_3.drop("Identifiant projet_")



#--7-- créer Projet Validé COEX ( RG dans dans neurone)

confition_proj_vali_coex = col("COEX : Décision Comité") == "Favorable"
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Projet Validé COEX", when(confition_proj_vali_coex, "Oui").otherwise("Non"))


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer nombre de Projet Validé COEX ( RG dans dans neurone)

confition_nb_proj_vali_coex = col("COEX : Décision Comité") == "Favorable"
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Nombre de Projet Validé COEX", when(confition_nb_proj_vali_coex, 1).otherwise(0))

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# renommer 'COEX : Date de validation du PV de présélection à Date de COEX ( RG )

F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("COEX : Date de validation du PV de présélection", "Date de COEX")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Mapper Instruire les dossiers à partir de PARAM_FG

instruire =PARAM_FG[["Catégorie","Instruire les dossiers HT"]]
F_G_projet_2_3 = F_G_projet_2_3.join(instruire, F_G_projet_2_3["Catégorie Dispositif"] == instruire["Catégorie"], "left_outer") 


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# renommer  'Date de signature client' à Date de contractualisation
F_G_projet_2_3 = F_G_projet_2_3.withColumnRenamed("Date de signature client", "Date de contractulisation")

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer date de fin de projet ( à voir avec le métier )
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer Nombre de projets contractualisés ( RG dans neurone)
confition_nb_proj_cont = col("Date de contractulisation").isNotNull()
F_G_projet_2_3 = F_G_projet_2_3.withColumn("Nombre de projets contractulisation", when(confition_nb_proj_cont, 1).otherwise(0))
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Mapper Accompagner les projets - Annéé 1 jusqu'a 10

accompagner =PARAM_FG[["Année","Catégorie","Accompagner les projets HT"]]


#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Accompagner les projets -total ( RG dans neurone)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer col Frais de gestion suivi par année ==> moyenne de toutes les colonnes Accompagner les projets
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer Nb Années de suivis ( RG dans neurone)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# créer accompagner les pojets -2021 jusqu'a pojets -2030 
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Créer Accompagner les projets -total
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------


F_G_projet_2_3 = F_G_projet_2_3.drop("Statut")

F_G_projet_2_3.show(truncate=False)

24/01/04 15:06:29 WARN TaskSetManager: Stage 84 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
24/01/04 15:06:30 WARN TaskSetManager: Stage 89 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+-------------------------------------+----------+-----------------+-------------------+----------------------+------------------------------------+-----------------------------+-------------------+-------------------------+----------------------+-------------------+-----------------------+----------------------------------+--------------------+------------+-----------------------+---------------+--------------+----------+-------------------------+------------------------------+----------------------------------+------------------+----------------------------+---------+-------------------------+-----------------------------------+
|Identifiant projet                   |Dispositif|AAP              |Acronyme du projet |Convention Projet SCAN|Instruction affectée autre opérateur|COPIL : Date de signature DPM|Date de COEX       |Date de contractulisation|COEX : Décision Comité|Date de relève     |COPIL : Décision Comité|Montant total de l'aide retenue HT|Catégorie Dispositif|Stade projet|S

In [17]:
F_G_projet_2_3_ = F_G_projet_2_3
F_G_projet_2_3_ = F_G_projet_2_3_.withColumn("Montant total de l'aide retenue HT", col("Montant total de l'aide retenue HT").cast("int"))
F_G_projet_2_3_ = F_G_projet_2_3_.withColumn("Instruire les dossiers HT", col("Instruire les dossiers HT").cast("int"))
F_G_projet_2_3_ = F_G_projet_2_3_.withColumn("Nombre de projets contractulisation", col("Nombre de projets contractulisation").cast("int"))


# Display the similar version of 

F_G_projet_2_3_fin = F_G_projet_2_3.groupBy("Identifiant projet").agg(
    first("Dispositif").alias("Dispositif"),
    first("AAP").alias("AAP"),
    first("Acronyme du projet").alias("Acronyme du projet"),
    first("Stade projet").alias("Stade projet"),
    first("Stade Relèves").alias("Stade Relèves"),
    first("Catégorie Dispositif").alias("Catégorie Dispositif"),
    first("Convention Projet SCAN").alias("Convention Projet SCAN"),
    first("Convention Projet pour FG").alias("Convention Projet pour FG"),
    #first("Instruction autre opérateur").alias("Instruction autre opérateur"),
    min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
    max("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
    first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
    max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
    first("Projet Validé COEX").alias("Projet Validé COEX"),
    first("Nombre de Projet Validé COEX").alias("Nombre de Projet Validé COEX"),  
    first("Date de COEX").alias("Date de COEX"),
    max("Instruire les dossiers HT").alias("Instruire les dossiers HT"),
    max("Date de contractulisation").alias("Date de contractulisation"),
    max("Nombre de projets contractulisation").alias("Nombre de projets contractulisation"),
)

In [18]:
F_G_projet_2_3.filter(col("Identifiant projet")=="I-DEMO-GENERIQUE 1-PXL-354").show(truncate=False)

24/01/04 15:06:31 WARN TaskSetManager: Stage 114 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
24/01/04 15:06:31 WARN TaskSetManager: Stage 118 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------+----------+-----------+------------------+----------------------+------------------------------------+-----------------------------+-------------------+-------------------------+----------------------+-------------------+-----------------------+----------------------------------+--------------------+------------+-----------------------+---------------+--------------+----------+-------------------------+------------------------------+----------------------------------+------------------+----------------------------+---------+-------------------------+-----------------------------------+
|Identifiant projet        |Dispositif|AAP        |Acronyme du projet|Convention Projet SCAN|Instruction affectée autre opérateur|COPIL : Date de signature DPM|Date de COEX       |Date de contractulisation|COEX : Décision Comité|Date de relève     |COPIL : Décision Comité|Montant total de l'aide retenue HT|Catégorie Dispositif|Stade projet|Stade Relèves          |Month de relè

In [19]:
F_G_projet_2_3_.filter(col("Identifiant projet")=="I-DEMO-GENERIQUE 1-PXL-354").show(truncate=False)

24/01/04 15:06:32 WARN TaskSetManager: Stage 154 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
24/01/04 15:06:32 WARN TaskSetManager: Stage 158 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------+----------+-----------+------------------+----------------------+------------------------------------+-----------------------------+-------------------+-------------------------+----------------------+-------------------+-----------------------+----------------------------------+--------------------+------------+-----------------------+---------------+--------------+----------+-------------------------+------------------------------+----------------------------------+------------------+----------------------------+---------+-------------------------+-----------------------------------+
|Identifiant projet        |Dispositif|AAP        |Acronyme du projet|Convention Projet SCAN|Instruction affectée autre opérateur|COPIL : Date de signature DPM|Date de COEX       |Date de contractulisation|COEX : Décision Comité|Date de relève     |COPIL : Décision Comité|Montant total de l'aide retenue HT|Catégorie Dispositif|Stade projet|Stade Relèves          |Month de relè

In [20]:
selected_columns = ["Dispositif", "AAP", "Stade projet"]
selected_df = F_G_projet_2_3_fin.select(selected_columns)
selected_df.show()

24/01/04 15:06:33 WARN TaskSetManager: Stage 194 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+------------+
|Dispositif|                 AAP|Stade projet|
+----------+--------------------+------------+
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    En cours|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|AERONEF BAS CARBONE1|    Terminés|
|   30 AERO|A

# F_G_Global_AAP

In [21]:
from pyspark.sql.functions import col, coalesce



F_G_Global_AAP = df[["Dispositif","AAP","Date de relève"]]

# renommer "Dispositif de sortie" à Dispositif
# Renommer AAP ==> Code AAP
# renommer "Date de relève" ==> 1er année de relève et choisir la date min
# renommer Catégorie Dispositif à Catégorie AAP

F_G_Global_AAP = F_G_Global_AAP.join(FG_Param2, F_G_Global_AAP["AAP"] == FG_Param2["_AAP"], "left_outer")
F_G_Global_AAP = F_G_Global_AAP.drop("_AAP")

# Mapper Procédure


#------------------------------------ Frais de gestion --------------------------------------------------------

# Créer Cadrage AAP ( RG dans neurone)
F_G_Global_AAP = F_G_Global_AAP.withColumn("Cadrer la procédure - AAP renouvelé", when((F_G_Global_AAP["AAP"].substr(-1, 1) != "1") & (F_G_Global_AAP["AAP"].substr(-1, 1) != "2"), 5500))
F_G_Global_AAP = F_G_Global_AAP.withColumn("Cadrer la procédure - AAP renouvelé", col("Cadrer la procédure - AAP renouvelé").cast("int"))
F_G_Global_AAP = F_G_Global_AAP.withColumn("Cadrer la procédure", when(F_G_Global_AAP["Cadrer la procédure - AAP renouvelé"].isNull(), lit(11167)))
F_G_Global_AAP = F_G_Global_AAP.withColumn("Cadrer la procédure", col("Cadrer la procédure").cast("int"))


F_G_Global_AAP = F_G_Global_AAP.withColumn("Cadrage AAP", coalesce(col("Cadrer la procédure"), col("Cadrer la procédure - AAP renouvelé")))
F_G_Global_AAP = F_G_Global_AAP.drop("Cadrer la procédure")
F_G_Global_AAP = F_G_Global_AAP.drop("Cadrer la procédure - AAP renouvelé")

# créer Phase receuil et préselection ( RG dans neurone )

selected_columns=["Dispositif de sortie","Recueillir les candidatures HT"]
selected_df = F_G_projet_1.select(selected_columns)
F_G_Global_AAP = F_G_Global_AAP.join(selected_df, F_G_Global_AAP["Dispositif"] == selected_df["Dispositif de sortie"], "left_outer")
F_G_Global_AAP = F_G_Global_AAP.drop("Dispositif de sortie")

# Phase instruction ( RG dans neurone )

"""col =["Dispositif","Instruire les dossiers HT"]
F_G_projet_2_3_fin_1 = F_G_projet_2_3_fin.select(col)
F_G_projet_2_3_fin_1 = F_G_projet_2_3_fin_1.withColumnRenamed("Dispositif", "Dispositif_")
F_G_Global_AAP = F_G_Global_AAP.join(F_G_projet_2_3_fin_1, F_G_Global_AAP["Dispositif"] == F_G_projet_2_3_fin_1["Dispositif_"], "left_outer")
F_G_Global_AAP = F_G_Global_AAP.drop("Dispositif_")"""


#Still working on it

# créer Phase suivi ( RG dans neurone)

# créer phase Suivi 2021
# créer phase Suivi 2022
# créer reporting ( RG dans neurone )

#------------------------------------ Multiplicateurs --------------------------------------------------------

# créer Nb eojets phase receuil et préselection ( RG dans neurone )



# créer Nb projets phase instruction ( RG dans neurone )
# créer Nb projets phase Suivi ( RG dans neurone )
# créer Nb Années ( RG dans neurone )
# créer Nb Experts dans Audition ( RG à confirmer )
# Mapper Nb expertises ( voir quel est la table de param à mapper)
# Nombre de projets décidés ( RG dans neurone )
# créer montant aides décidées ( RG dans neurone )
# créer FG par projet décidés ( RG dans neurone)
# créer % 


F_G_Global_AAP.show()
# a regarder dans neurone



24/01/04 15:06:33 WARN TaskSetManager: Stage 206 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
24/01/04 15:06:33 WARN TaskSetManager: Stage 208 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+----------+-----------------+-------------------+-------------+-----------+------------------------------+
|Dispositif|              AAP|     Date de relève|Catégorie_AAP|Cadrage AAP|Recueillir les candidatures HT|
+----------+-----------------+-------------------+-------------+-----------+------------------------------+
| 30 METAUX|DEPEND MET CRIT 1|2022-04-05 00:00:00|    SA/FR2030|      11167|                          1416|
| 30 METAUX|DEPEND MET CRIT 1|2022-04-05 00:00:00|    SA/FR2030|      11167|                          NULL|
| 30 METAUX|DEPEND MET CRIT 1|2022-04-05 00:00:00|    SA/FR2030|      11167|                          1416|
| 30 METAUX|DEPEND MET CRIT 1|2022-04-05 00:00:00|    SA/FR2030|      11167|                          1416|
| 30 METAUX|DEPEND MET CRIT 1|2022-04-05 00:00:00|    SA/FR2030|      11167|                          1416|
| 30 METAUX|DEPEND MET CRIT 1|2022-04-05 00:00:00|    SA/FR2030|      11167|                          1416|
| 30 METAUX|DEPEND MET CRIT 

In [22]:
# Display the similar version of 

F_G_Global_AAP_test = F_G_Global_AAP.groupBy("Dispositif","AAP","Catégorie_AAP").agg(
    first("AAP").alias("AAP"),
    min("Date de relève").alias("Date de relève"),
    first("Catégorie_AAP").alias("Catégorie_AAP"),
    first("Cadrage AAP").alias("Cadrage AAP"),
    sum("Recueillir les candidatures HT").alias("Phase receuil et préselection"),)

In [23]:
F_G_Global_AAP_test.filter(col("Dispositif")=="30 AUTO").show()

24/01/04 15:06:34 WARN TaskSetManager: Stage 225 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.
24/01/04 15:06:34 WARN TaskSetManager: Stage 227 contains a task of very large size (16886 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------------+-------------+--------------------+-------------------+-------------+-----------+-----------------------------+
|Dispositif|                 AAP|Catégorie_AAP|                 AAP|     Date de relève|Catégorie_AAP|Cadrage AAP|Phase receuil et préselection|
+----------+--------------------+-------------+--------------------+-------------------+-------------+-----------+-----------------------------+
|   30 AUTO|          CORAM 2022|    SA/FR2030|          CORAM 2022|2022-06-09 00:00:00|    SA/FR2030|      11167|                     63051648|
|   30 AUTO|DIV. SOUSTRAI AUTO 1|    SA/FR2030|DIV. SOUSTRAI AUTO 1|2022-03-15 00:00:00|    SA/FR2030|      11167|                    128844672|
|   30 AUTO|      Gré-à-gré Etat|         NULL|      Gré-à-gré Etat|2021-05-01 00:00:00|         NULL|       5500|                      7538784|
|   30 AUTO|       INVEST AUTO 1|    SA/FR2030|       INVEST AUTO 1|2023-03-29 00:00:00|    SA/FR2030|      11167|                

In [24]:
# Display the similar version of 

F_G_Global_convention = F_G_projet_2_3.groupBy("Dispositif").agg(
    first("AAP").alias("AAP"),
    first("Acronyme du projet").alias("Acronyme du projet"),
    first("Stade projet").alias("Stade projet"),
    first("Stade Relèves").alias("Stade Relèves"),
    first("Catégorie Dispositif").alias("Catégorie Dispositif"),
    first("Convention Projet SCAN").alias("Convention Projet SCAN"),
    first("Convention Projet pour FG").alias("Convention Projet pour FG"),
    #first("Instruction autre opérateur").alias("Instruction autre opérateur"),
    min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
    sum("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
    first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
    max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
    first("Projet Validé COEX").alias("Projet Validé COEX"),
    first("Nombre de Projet Validé COEX").alias("Nombre de Projet Validé COEX"),  
    first("Date de COEX").alias("Date de COEX"),
    max("Instruire les dossiers HT").alias("Instruire les dossiers HT"),
    max("Date de contractulisation").alias("Date de contractulisation"),
)

In [25]:
F_G_projet_last =F_G_projet_2_3[['Dispositif',
 'AAP',
 'Date de relève']]

In [26]:
def F_G_projet_2_3_(df):
    F_G_projet_2_3_ = F_G_projet_2_3_
    return df

In [27]:
from pyspark.sql import functions as F

def FDG_projet_2_3_(df):
    result_df = df.groupBy("Identifiant projet").agg(
        F.first("Dispositif").alias("Dispositif"),
        F.first("AAP").alias("AAP"),
        F.first("Acronyme du projet").alias("Acronyme du projet"),
        F.first("Stade projet").alias("Stade projet"),
        F.first("Stade Relèves").alias("Stade Relèves"),
        F.first("Catégorie Dispositif").alias("Catégorie Dispositif"),
        F.first("Convention Projet SCAN").alias("Convention Projet SCAN"),
        F.first("Convention Projet pour FG").alias("Convention Projet pour FG"),
        # F.first("Instruction autre opérateur").alias("Instruction autre opérateur"),
        F.min("COPIL : Date de signature DPM").alias("COPIL : Date de signature DPM"),
        F.max("Montant total de l'aide retenue HT").alias("Montant total de l'aide retenue HT"),
        F.first("Nombre de projet décidés COPIL").alias("Nombre de projet décidés COPIL"),
        F.max("Nombre de partenaire décidés COPIL").alias("Nombre de partenaire décidés COPIL"),
        F.first("Projet Validé COEX").alias("Projet Validé COEX"),
        F.first("Nombre de Projet Validé COEX").alias("Nombre de Projet Validé COEX"),  
        F.first("Date de COEX").alias("Date de COEX"),
        F.max("Instruire les dossiers HT").alias("Instruire les dossiers HT"),
        F.max("Date de contractulisation").alias("Date de contractulisation"),
        F.max("Nombre de projets contractulisation").alias("Nombre de projets contractulisation"),
    )

    return result_df